In [3]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
# import tmdb

## ////////////////////////////
# 2018
## ///////////////////////////

In [4]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2018"
df1 = pd.read_html(link, header=0)[2]
df2 = pd.read_html(link, header=0)[3]
df3 = pd.read_html(link, header=0)[4]
df4 = pd.read_html(link, header=0)[5]

In [5]:
df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)

In [6]:
df.head(2)

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.,Unnamed: 6
0,JANUARY,5,Insidious: The Last Key,Universal Pictures / Blumhouse Productions / S...,Adam Robitel (director); Leigh Whannell (scree...,[2],NaN
1,JANUARY,5,The Strange Ones,Vertical Entertainment,Lauren Wolkstein (director); Christopher Radcl...,[3],NaN


In [7]:
# !pip install tmdbv3api

In [8]:
from tmdbv3api import TMDb
import json
import requests
tmdb = TMDb()
tmdb.api_key = '3163424ed866819ce9c06db3ad13497e'

In [9]:
from tmdbv3api import Movie
tmdb_movie = Movie()
def get_genre(x):
    genres = []
    result = tmdb_movie.search(x)
    movie_id = result[0].id
    response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
    data_json = response.json()
    if data_json['genres']:
        genre_str = " " 
        for i in range(0,len(data_json['genres'])):
            if data_json['genres'][i]['name'] == 'TV Movie':
                continue
            else:
                genres.append(data_json['genres'][i]['name'])
        return genre_str.join(genres)
    else:
        np.NaN

In [10]:
df['genres'] = df['Title'].map(lambda x: get_genre(str(x)))

In [11]:
df.head(2)

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.,Unnamed: 6,genres
0,JANUARY,5,Insidious: The Last Key,Universal Pictures / Blumhouse Productions / S...,Adam Robitel (director); Leigh Whannell (scree...,[2],NaN,Mystery Horror Thriller
1,JANUARY,5,The Strange Ones,Vertical Entertainment,Lauren Wolkstein (director); Christopher Radcl...,[3],NaN,Thriller Drama


In [12]:
df_2018 = df[['Title','Cast and crew','genres']]
df_2018.head()

,Title,Cast and crew,genres
0,Insidious: The Last Key,Adam Robitel (director); Leigh Whannell (scree...,Mystery Horror Thriller
1,The Strange Ones,Lauren Wolkstein (director); Christopher Radcl...,Thriller Drama
2,Stratton,"Simon West (director); Duncan Falconer, Warren...",Action Thriller
3,Sweet Country,"Warwick Thornton (director); David Tranter, St...",Drama History Western
4,The Commuter,Jaume Collet-Serra (director); Byron Willinger...,Action Thriller


In [13]:
g = []
for i in df_2018.genres:
    w = []
    try:
        w = i.split(' ')
    except:
        w.append(' ')
    for j in w:
        if j not in g:
            g.append(j)
print(g)

['Mystery', 'Horror', 'Thriller', 'Drama', 'Action', 'History', 'Western', 'Crime', 'Comedy', 'Fantasy', 'Adventure', 'Animation', 'War', 'Romance', 'Music', 'Science', 'Fiction', 'Family', ' ', 'Documentary']


In [14]:
def get_director(x):
    return x.split(' (director')[0]

In [15]:
def get_actor1(x):
    return (x.split('screenplay); ')[-1]).split(', ')[0]


def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return (' ')
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])
        
           
def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return (' ')
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [16]:
df_2018['actor_1_name'] = df_2018['Cast and crew'].map(lambda x:get_actor1(x))
df_2018['actor_2_name'] = df_2018['Cast and crew'].map(lambda x:get_actor2(x))
df_2018['actor_3_name'] = df_2018['Cast and crew'].map(lambda x:get_actor3(x))
df_2018['director_name'] = df_2018['Cast and crew'].map(lambda x:get_director(x))
df_2018.head()

,Title,Cast and crew,genres,actor_1_name,actor_2_name,actor_3_name,director_name
0,Insidious: The Last Key,Adam Robitel (director); Leigh Whannell (scree...,Mystery Horror Thriller,Lin Shaye,Angus Sampson,Leigh Whannell,Adam Robitel
1,The Strange Ones,Lauren Wolkstein (director); Christopher Radcl...,Thriller Drama,Alex Pettyfer,James Freedson-Jackson,Emily Althaus,Lauren Wolkstein
2,Stratton,"Simon West (director); Duncan Falconer, Warren...",Action Thriller,Dominic Cooper,Austin Stowell,Gemma Chan,Simon West
3,Sweet Country,"Warwick Thornton (director); David Tranter, St...",Drama History Western,Bryan Brown,Sam Neill,,Warwick Thornton
4,The Commuter,Jaume Collet-Serra (director); Byron Willinger...,Action Thriller,Liam Neeson,Vera Farmiga,Patrick Wilson,Jaume Collet-Serra


movie_title	director_name	actor_1_name	actor_2_name	actor_3_name	genres	combined

In [17]:
df_2018 = df_2018.rename(columns={'Title':'movie_title', })
df_2018.drop('Cast and crew',axis=1,inplace=True)
df_2018 = df_2018.apply(lambda x: x.str.lower())
# df_2018.head(2)

In [18]:
df_2018['combined'] = df_2018.director_name +' '+ df_2018.actor_1_name + ' ' + df_2018.actor_2_name + ' ' + df_2018.actor_3_name + ' ' + df_2018.genres

In [19]:
new_df18 = df_2018[['movie_title','director_name', 'actor_1_name', 'actor_2_name', 'actor_3_name', 'genres', 'combined']]

In [20]:
new_df18.head(2)

,movie_title,director_name,actor_1_name,actor_2_name,actor_3_name,genres,combined
0,insidious: the last key,adam robitel,lin shaye,angus sampson,leigh whannell,mystery horror thriller,adam robitel lin shaye angus sampson leigh wha...
1,the strange ones,lauren wolkstein,alex pettyfer,james freedson-jackson,emily althaus,thriller drama,lauren wolkstein alex pettyfer james freedson-...


## ///////////////////

#                                         2019

## //////////////

In [21]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2019"
df1 = pd.read_html(link, header=0)[3]
df2 = pd.read_html(link, header=0)[4]
df3 = pd.read_html(link, header=0)[5]
df4 = pd.read_html(link, header=0)[6]

In [22]:
dff = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)

In [23]:
dff.head(2)

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,NaN,4,Escape Room,Columbia Pictures,"Adam Robitel (director); Bragi F. Schut, Maria...",[2]
1,NaN,4,Rust Creek,IFC Films,Jen McGowan (director); Julie Lipson (screenpl...,[3]


In [24]:
dff['genres'] = dff['Title'].map(lambda x: get_genre(str(x)))

In [25]:
df_2019 = dff[['Title','Cast and crew','genres']]
df_2019.head(2)

,Title,Cast and crew,genres
0,Escape Room,"Adam Robitel (director); Bragi F. Schut, Maria...",Thriller Action Mystery Adventure Horror
1,Rust Creek,Jen McGowan (director); Julie Lipson (screenpl...,Thriller Drama


In [26]:
df_2019[df_2019['Title']=='insidious: the last key']

,Title,Cast and crew,genres


In [27]:
# df_2019['Cast and crew']

In [28]:
df_2019['actor_1_name'] = df_2019['Cast and crew'].map(lambda x:get_actor1(x))
df_2019['actor_2_name'] = df_2019['Cast and crew'].map(lambda x:get_actor2(x))
df_2019['actor_3_name'] = df_2019['Cast and crew'].map(lambda x:get_actor3(x))
df_2019['director_name'] = df_2019['Cast and crew'].map(lambda x:get_director(x))
df_2019.head()

,Title,Cast and crew,genres,actor_1_name,actor_2_name,actor_3_name,director_name
0,Escape Room,"Adam Robitel (director); Bragi F. Schut, Maria...",Thriller Action Mystery Adventure Horror,Taylor Russell,Logan Miller,Deborah Ann Woll,Adam Robitel
1,Rust Creek,Jen McGowan (director); Julie Lipson (screenpl...,Thriller Drama,Hermione Corfield,Jay Paulson,Sean O'Bryan,Jen McGowan
2,American Hangman,Wilson Coneybeare (director/screenplay); Donal...,Thriller,Donald Sutherland,Vincent Kartheiser,Oliver Dennis,Wilson Coneybeare
3,A Dog's Way Home,Charles Martin Smith (director); W. Bruce Came...,Drama Adventure Family,Bryce Dallas Howard,Edward James Olmos,Alexandra Shipp,Charles Martin Smith
4,The Upside,Neil Burger (director); Jon Hartmere (screenpl...,Comedy Drama,Bryan Cranston,Kevin Hart,Nicole Kidman,Neil Burger


In [29]:
# df_2019.drop('Cast and crew',axis=1,inplace=True)

In [30]:
df_2019 = df_2019.apply(lambda x:x.str.lower())
df_2019 = df_2019.rename(columns={'Title':'movie_title'})

In [31]:
df_2019[df_2019['movie_title']=='insidious: the last key']

,movie_title,Cast and crew,genres,actor_1_name,actor_2_name,actor_3_name,director_name


In [32]:
df_2019['combined'] = df_2019.director_name +' '+ df_2019.actor_1_name + ' ' + df_2019.actor_2_name + ' ' + df_2019.actor_3_name + ' ' + df_2019.genres

In [33]:
new_df19 = df_2019[['movie_title','director_name', 'actor_1_name', 'actor_2_name', 'actor_3_name', 'genres', 'combined']]

In [34]:
new_df19.head(2)

,movie_title,director_name,actor_1_name,actor_2_name,actor_3_name,genres,combined
0,escape room,adam robitel,taylor russell,logan miller,deborah ann woll,thriller action mystery adventure horror,adam robitel taylor russell logan miller debor...
1,rust creek,jen mcgowan,hermione corfield,jay paulson,sean o'bryan,thriller drama,jen mcgowan hermione corfield jay paulson sean...


In [35]:
new_df18.isna().sum()

movie_title      0
director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           3
combined         3
dtype: int64

In [36]:
new_df19.isna().sum()

movie_title      0
director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           1
combined         1
dtype: int64

In [37]:
my_df = new_df18.append(new_df19,ignore_index=True)

In [38]:
my_df.isnull().sum()

movie_title      0
director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           4
combined         4
dtype: int64

In [39]:
my_df.shape

(508, 7)

## /////////////////////////
#  2020
## ///////////////////////

In [40]:
link = 'https://en.wikipedia.org/wiki/List_of_American_films_of_2020'
df1 = pd.read_html(link,header=0)[3]
df2 = pd.read_html(link,header=0)[4]
df2 = pd.read_html(link,header=0)[5]
df2 = pd.read_html(link,header=0)[6]
df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)
df.head(2)

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,JANUARY,3,The Grudge,Screen Gems / Stage 6 Films / Ghost House Pict...,Nicolas Pesce (director/screenplay); Andrea Ri...,[2]
1,JANUARY,10,Underwater,20th Century Fox / TSG Entertainment / Chernin...,"William Eubank (director); Brian Duffield, Ada...",[3]


In [41]:
df = df[['Title','Cast and crew']]

In [42]:
df['genres'] = df['Title'].map(lambda x : get_genre(str(x)))

In [43]:
df.head(2)

,Title,Cast and crew,genres
0,The Grudge,Nicolas Pesce (director/screenplay); Andrea Ri...,Horror Mystery Thriller
1,Underwater,"William Eubank (director); Brian Duffield, Ada...",Action Horror Science Fiction Thriller


In [44]:
df.dropna(inplace=True)

In [45]:
df.isna().sum()

Title            0
Cast and crew    0
genres           0
dtype: int64

In [46]:
df['actor_1_name'] = df['Cast and crew'].map(lambda x:get_actor1(x))
df['actor_2_name'] = df['Cast and crew'].map(lambda x:get_actor2(x))
df['actor_3_name'] = df['Cast and crew'].map(lambda x:get_actor3(x))
df['director_name'] = df['Cast and crew'].map(lambda x:get_director(x))
df.head()

,Title,Cast and crew,genres,actor_1_name,actor_2_name,actor_3_name,director_name
0,The Grudge,Nicolas Pesce (director/screenplay); Andrea Ri...,Horror Mystery Thriller,Andrea Riseborough,Demián Bichir,John Cho,Nicolas Pesce
1,Underwater,"William Eubank (director); Brian Duffield, Ada...",Action Horror Science Fiction Thriller,Kristen Stewart,Vincent Cassel,Jessica Henwick,William Eubank
2,Like a Boss,"Miguel Arteta (director); Sam Pitman, Adam Col...",Comedy,Tiffany Haddish,Rose Byrne,Salma Hayek,Miguel Arteta
3,Inherit the Viper,Anthony Jerjen (director); Andrew Crabtree (sc...,Drama Thriller Crime,Josh Hartnett,Margarita Levieva,Chandler Riggs,Anthony Jerjen
4,The Sonata,Andrew Desmond (director/screenplay); Arthur M...,Horror Thriller Mystery,Freya Tingley,Simon Abkarian,Rutger Hauer,Andrew Desmond


In [47]:
df_2020 = df[['Title','director_name','actor_1_name','actor_2_name','actor_3_name','genres']]
df_2020.head(2)

,Title,director_name,actor_1_name,actor_2_name,actor_3_name,genres
0,The Grudge,Nicolas Pesce,Andrea Riseborough,Demián Bichir,John Cho,Horror Mystery Thriller
1,Underwater,William Eubank,Kristen Stewart,Vincent Cassel,Jessica Henwick,Action Horror Science Fiction Thriller


In [48]:
df_2020 = df_2020.apply(lambda x: x.str.lower())

In [49]:
df_2020.head(2)

,Title,director_name,actor_1_name,actor_2_name,actor_3_name,genres
0,the grudge,nicolas pesce,andrea riseborough,demián bichir,john cho,horror mystery thriller
1,underwater,william eubank,kristen stewart,vincent cassel,jessica henwick,action horror science fiction thriller


In [50]:
df_2020.isna().sum()

Title            0
director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           0
dtype: int64

In [51]:
df_2020.rename(columns={'Title':'movie_title'},inplace=True)

In [52]:
df_2020['combined'] = df_2020.director_name + ' ' +  df_2020.actor_1_name + ' '+df_2020.actor_2_name + ' ' + df_2020.actor_3_name + ' ' + df_2020.genres

In [53]:
df181920 = my_df.append(df_2020,ignore_index=True)

In [54]:
df_2020.columns

Index(['movie_title', 'director_name', 'actor_1_name', 'actor_2_name',
       'actor_3_name', 'genres', 'combined'],
      dtype='object')

In [55]:
df181920

,movie_title,director_name,actor_1_name,actor_2_name,actor_3_name,genres,combined
0,insidious: the last key,adam robitel,lin shaye,angus sampson,leigh whannell,mystery horror thriller,adam robitel lin shaye angus sampson leigh wha...
1,the strange ones,lauren wolkstein,alex pettyfer,james freedson-jackson,emily althaus,thriller drama,lauren wolkstein alex pettyfer james freedson-...
2,stratton,simon west,dominic cooper,austin stowell,gemma chan,action thriller,simon west dominic cooper austin stowell gemma...
3,sweet country,warwick thornton,bryan brown,sam neill,,drama history western,warwick thornton bryan brown sam neill drama...
4,the commuter,jaume collet-serra,liam neeson,vera farmiga,patrick wilson,action thriller,jaume collet-serra liam neeson vera farmiga pa...
...,...,...,...,...,...,...,...
760,little women,greta gerwig,saoirse ronan,emma watson,florence pugh,drama romance,greta gerwig saoirse ronan emma watson florenc...
761,1917,sam mendes,george mackay,dean-charles chapman,mark strong,war drama,sam mendes george mackay dean-charles chapman ...
762,just mercy,destin daniel cretton,michael b. jordan,jamie foxx,brie larson,drama,destin daniel cretton michael b. jordan jamie ...
763,clemency,chinonye chukwu,alfre woodard,wendell pierce,aldis hodge,drama,chinonye chukwu alfre woodard wendell pierce a...


In [56]:
previousData = pd.read_csv('new_data.csv')
previousData.head(2)

,movie_title,director_name,actor_1_name,actor_2_name,actor_3_name,genres,combined
0,monsters vs. aliens,rob letterman,amy poehler,rainn wilson,stephen colbert,action adventure animation comedy family sci-fi,rob letterman amy poehler rainn wilson stephen...
1,puss in boots,chris miller,salma hayek,constance marie,amy sedaris,action adventure animation comedy family fantasy,chris miller salma hayek constance marie amy s...


In [57]:
previousData.shape

(10795, 7)

In [58]:
FinalData = df181920.append(previousData)
FinalData.shape

(11560, 7)

In [59]:
FinalData.isna().sum()

movie_title      0
director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           4
combined         4
dtype: int64

In [60]:
FinalData.dropna(inplace=True)

In [61]:
FinalData.isna().sum()

movie_title      0
director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           0
combined         0
dtype: int64

In [62]:
FinalData.columns

Index(['movie_title', 'director_name', 'actor_1_name', 'actor_2_name',
       'actor_3_name', 'genres', 'combined'],
      dtype='object')

In [63]:
FinalData.drop_duplicates(subset='movie_title',keep='last',inplace=True)
FinalData.shape

(11382, 7)

In [64]:
FinalData.reset_index(inplace = True)

In [65]:
FinalData.drop('index',inplace=True,axis=1)

In [66]:
FinalData

,movie_title,director_name,actor_1_name,actor_2_name,actor_3_name,genres,combined
0,insidious: the last key,adam robitel,lin shaye,angus sampson,leigh whannell,mystery horror thriller,adam robitel lin shaye angus sampson leigh wha...
1,the strange ones,lauren wolkstein,alex pettyfer,james freedson-jackson,emily althaus,thriller drama,lauren wolkstein alex pettyfer james freedson-...
2,stratton,simon west,dominic cooper,austin stowell,gemma chan,action thriller,simon west dominic cooper austin stowell gemma...
3,sweet country,warwick thornton,bryan brown,sam neill,,drama history western,warwick thornton bryan brown sam neill drama...
4,the commuter,jaume collet-serra,liam neeson,vera farmiga,patrick wilson,action thriller,jaume collet-serra liam neeson vera farmiga pa...
...,...,...,...,...,...,...,...
11377,an american vampire story,luis esteban\t,trevor lissauer,johnny venocur,adam west,comedy horror,luis esteban\t trevor lissauer johnny venocur ...
11378,fit to kill,andy sidaris,dona speir,roberta vasquez,bruce penhall,action romance thriller,andy sidaris dona speir roberta vasquez bruce ...
11379,cadet kelly,larry shaw,hilary duff,christy carlson romano,gary cole,comedy,larry shaw hilary duff christy carlson romano ...
11380,robin hood,john irvin,patrick bergin,uma thurman,david morrissey,drama action romance,john irvin patrick bergin uma thurman david mo...


In [67]:
FinalData.to_csv('FinalData.csv',index=False)

In [68]:
import pickle
with open('FinalData_p', 'wb') as f1:
    pickle.dump(FinalData, f1)

# ALL ENGLISH MOVIES IN THIS FILE